In [1]:
import pandas as pd
import nibabel as nib
from os.path import join, exists

df = pd.read_csv('PET_AUTOPSY.csv')

In [2]:
for indx_, row_ in df.iterrows():
    
    ad_pathology = row_['ALZHEIMER PATHOLOGY']
    lb_pathology = row_['LEWY PATHOLOGY']
    
    if ad_pathology > 1 and lb_pathology <= 1:
        
        df.loc[indx_,'GRUPO'] = 'pure-AD'
    
    elif ad_pathology > 1 and lb_pathology > 1:
        
        df.loc[indx_,'GRUPO'] = 'Mixed'
    
    elif ad_pathology <= 1 and lb_pathology > 1:
        
        df.loc[indx_,'GRUPO'] = 'pure-LB'
    
    else:
        
        df.loc[indx_,'GRUPO'] = 'Negative'

df = df.loc[df['GRUPO'] != 'Negative']
df = df.loc[df['GRUPO'] != 'Mixed']

df
        

,SUBJECT_ID,ORIG_GROUP,LAST_DIAGNOSIS,FDG_ID,FDG_AGE,AGE_DEATH,ALZHEIMER PATHOLOGY,LEWY PATHOLOGY,GRUPO
0,003_S_4892,AD,AD,334485,75.3,76,3,0,pure-AD
1,005_S_0448,MCI,AD,226091,90.6,93,1,2,pure-LB
4,011_S_0008,CN,MCI,26885,85.6,95,1,3,pure-LB
6,011_S_0053,AD,AD,83975,82.2,86,1,3,pure-LB
9,011_S_0362,MCI,AD,141405,73.6,75,3,0,pure-AD
10,011_S_0861,MCI,AD,158134,90.3,93,3,0,pure-AD
13,011_S_4827,AD,AD,906469,76.2,78,3,0,pure-AD
14,014_S_0563,MCI,AD,196278,85.1,93,1,2,pure-LB
16,014_S_4058,LMCI,AD,246832,84.8,90,3,0,pure-AD
18,014_S_4615,AD,AD,301226,86.7,92,3,0,pure-AD


In [20]:
import numpy as np

dir_patients = '/Users/jsilva/data/MIA/Autopsia'

atlas_img = nib.load('cortex_Harvard_Oxford_orig_atlas.nii.gz')
atlas_data = atlas_img.get_fdata()
atlas_csv = 'Harvard_Oxford.csv'
atlas_df = pd.read_csv(atlas_csv)


for indx_, row_ in df.iterrows():
    
    subject_id = row_['SUBJECT_ID']
    img_id = row_['FDG_ID']
    pet_name = join(dir_patients, f'{subject_id}_I{img_id}', 'swfdg_normhist.hdr')
    #print(pet_name)
    if exists(pet_name):
        img = nib.load(pet_name)
        data = img.get_fdata()
        
        for indx_atlas, row_atlas in atlas_df.iterrows():
            roi_name = row_atlas['ROI_NAME']
            roi_n = row_atlas['ROI_NUM']
            where_roi = np.where(atlas_data == roi_n)
            
            mean_roi = data[where_roi].mean()
            df.loc[indx_,roi_name] = mean_roi
            
df        

,SUBJECT_ID,ORIG_GROUP,LAST_DIAGNOSIS,FDG_ID,FDG_AGE,AGE_DEATH,ALZHEIMER PATHOLOGY,LEWY PATHOLOGY,GRUPO,Temporal_Pole,...,Parietal_Operculum_Cortex,Planum_Polare,Heschl's_Gyrus,Planum_Temporale,Occipital_Pole,Thalamus,Striatum,Brainstem,Hippocampus,Amygdala
0,003_S_4892,AD,AD,334485,75.3,76,3,0,pure-AD,0.905302,...,1.284167,1.206530,1.423608,1.330791,1.482770,1.121088,1.175532,0.895746,1.020664,0.982098
1,005_S_0448,MCI,AD,226091,90.6,93,1,2,pure-LB,0.994052,...,1.435870,1.208496,1.565999,1.513874,1.271314,0.973781,1.330373,1.010803,1.072394,1.120139
4,011_S_0008,CN,MCI,26885,85.6,95,1,3,pure-LB,1.052362,...,1.260794,1.280594,1.537939,1.470419,1.404669,1.211122,1.256367,0.925326,1.077617,1.050791
6,011_S_0053,AD,AD,83975,82.2,86,1,3,pure-LB,1.010989,...,1.255864,1.249175,1.501377,1.257903,1.298316,1.013251,1.254987,1.012383,1.136809,1.129274
9,011_S_0362,MCI,AD,141405,73.6,75,3,0,pure-AD,0.899030,...,1.302902,1.347796,1.598650,1.564037,1.529169,1.170500,1.245975,0.973428,1.024073,1.029125
10,011_S_0861,MCI,AD,158134,90.3,93,3,0,pure-AD,0.788074,...,1.332653,1.119537,1.389431,1.373873,1.449426,0.808612,0.984717,0.910093,0.997777,1.001854
13,011_S_4827,AD,AD,906469,76.2,78,3,0,pure-AD,0.799955,...,1.349063,1.014870,1.398580,1.412917,1.590368,0.996838,1.182004,0.843337,0.875962,0.893632
14,014_S_0563,MCI,AD,196278,85.1,93,1,2,pure-LB,0.922785,...,1.317961,1.136658,1.298665,1.256327,1.514056,0.933598,1.197445,0.984119,0.902683,1.037154
16,014_S_4058,LMCI,AD,246832,84.8,90,3,0,pure-AD,0.936481,...,1.304179,1.190423,1.468963,1.465101,1.679486,1.116555,1.134851,0.967362,1.017832,1.020538
18,014_S_4615,AD,AD,301226,86.7,92,3,0,pure-AD,0.921282,...,1.244800,1.046732,1.202993,1.236580,1.400050,0.965332,1.081781,0.968254,0.879256,0.971064


In [21]:
df_ad = df.loc[df['GRUPO'] == 'pure-AD']
df_lewy = df.loc[df['GRUPO'] == 'pure-LB']


In [22]:
df_lewy

,SUBJECT_ID,ORIG_GROUP,LAST_DIAGNOSIS,FDG_ID,FDG_AGE,AGE_DEATH,ALZHEIMER PATHOLOGY,LEWY PATHOLOGY,GRUPO,Temporal_Pole,...,Parietal_Operculum_Cortex,Planum_Polare,Heschl's_Gyrus,Planum_Temporale,Occipital_Pole,Thalamus,Striatum,Brainstem,Hippocampus,Amygdala
1,005_S_0448,MCI,AD,226091,90.6,93,1,2,pure-LB,0.994052,...,1.435870,1.208496,1.565999,1.513874,1.271314,0.973781,1.330373,1.010803,1.072394,1.120139
4,011_S_0008,CN,MCI,26885,85.6,95,1,3,pure-LB,1.052362,...,1.260794,1.280594,1.537939,1.470419,1.404669,1.211122,1.256367,0.925326,1.077617,1.050791
6,011_S_0053,AD,AD,83975,82.2,86,1,3,pure-LB,1.010989,...,1.255864,1.249175,1.501377,1.257903,1.298316,1.013251,1.254987,1.012383,1.136809,1.129274
14,014_S_0563,MCI,AD,196278,85.1,93,1,2,pure-LB,0.922785,...,1.317961,1.136658,1.298665,1.256327,1.514056,0.933598,1.197445,0.984119,0.902683,1.037154
28,027_S_0408,MCI,AD,385375,89.4,91,1,2,pure-LB,0.847555,...,1.355567,1.095675,1.322183,1.430719,1.323738,0.911435,1.059663,0.832240,0.729189,0.762369
30,027_S_4936,LMCI,AD,345462,78.2,81,1,3,pure-LB,1.030225,...,1.133781,1.152533,1.236924,1.202057,1.160946,1.085487,1.311177,0.906953,1.016181,1.051593
43,033_S_0723,MCI,AD,153173,82.2,83,1,3,pure-LB,0.963807,...,1.261751,1.134905,1.370014,1.272829,0.952076,1.015593,1.199414,0.923604,1.037120,1.067219
60,114_S_1118,MCI,MCI,406049,89.7,91,1,2,pure-LB,0.925634,...,1.253064,1.058093,1.361198,1.301235,1.170973,0.887542,1.133581,0.919304,0.921853,1.072172


In [24]:
from scipy.stats import ttest_ind

for indx_, row_ in atlas_df.iterrows():
    
    roi_name = row_['ROI_NAME']
    t_val, p_val = ttest_ind(df_ad[roi_name], df_lewy[roi_name], axis=0, equal_var=True)
    
    if p_val < 0.005:
        print(f"{roi_name}: {p_val:.3f}")
    
    else:
        df.drop(roi_name, axis=1, inplace=True)
        
df    

Inferior_Temporal_Gyrus_anterior_division: 0.001
Intracalcarine_Cortex: 0.002
Supracalcarine_Cortex: 0.005


,SUBJECT_ID,ORIG_GROUP,LAST_DIAGNOSIS,FDG_ID,FDG_AGE,AGE_DEATH,ALZHEIMER PATHOLOGY,LEWY PATHOLOGY,GRUPO,Inferior_Temporal_Gyrus_anterior_division,Intracalcarine_Cortex,Supracalcarine_Cortex
0,003_S_4892,AD,AD,334485,75.3,76,3,0,pure-AD,1.082464,1.806612,1.867905
1,005_S_0448,MCI,AD,226091,90.6,93,1,2,pure-LB,1.251840,1.562329,1.582445
4,011_S_0008,CN,MCI,26885,85.6,95,1,3,pure-LB,1.169544,1.659618,1.659314
6,011_S_0053,AD,AD,83975,82.2,86,1,3,pure-LB,1.314990,1.516506,1.496108
9,011_S_0362,MCI,AD,141405,73.6,75,3,0,pure-AD,1.059917,1.709761,1.648271
10,011_S_0861,MCI,AD,158134,90.3,93,3,0,pure-AD,1.135093,1.804450,1.632121
13,011_S_4827,AD,AD,906469,76.2,78,3,0,pure-AD,1.075888,1.817859,1.800605
14,014_S_0563,MCI,AD,196278,85.1,93,1,2,pure-LB,1.185621,1.575078,1.542219
16,014_S_4058,LMCI,AD,246832,84.8,90,3,0,pure-AD,1.133862,1.869121,1.797530
18,014_S_4615,AD,AD,301226,86.7,92,3,0,pure-AD,1.135471,1.708735,1.623819


In [31]:
X = df.iloc[:,-3:]

y = df['GRUPO']

X

,Inferior_Temporal_Gyrus_anterior_division,Intracalcarine_Cortex,Supracalcarine_Cortex
0,1.082464,1.806612,1.867905
1,1.251840,1.562329,1.582445
4,1.169544,1.659618,1.659314
6,1.314990,1.516506,1.496108
9,1.059917,1.709761,1.648271
10,1.135093,1.804450,1.632121
13,1.075888,1.817859,1.800605
14,1.185621,1.575078,1.542219
16,1.133862,1.869121,1.797530
18,1.135471,1.708735,1.623819


In [41]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score

model = LogisticRegression(penalty='l2', dual=False, tol=0.0001, C=1.0, fit_intercept=True, intercept_scaling=1, class_weight="balanced", random_state=None, solver='lbfgs', max_iter=100, verbose=0, warm_start=False, n_jobs=None, l1_ratio=None)

model.fit(X,y)

LogisticRegression(class_weight='balanced')

In [42]:
y_pred = model.predict(X)
conf_matrix = confusion_matrix(y, y_pred)
accuracy_score = accuracy_score(y, y_pred)
print(accuracy_score)
conf_matrix

0.8611111111111112


array([[24,  4],
       [ 1,  7]])

In [43]:
from sklearn.svm import SVC

model = SVC(C=1.0, kernel='rbf', degree=3, gamma='scale', coef0=0.0, shrinking=True, probability=False, tol=0.001, cache_size=200, class_weight="balanced", verbose=False, max_iter=-1, decision_function_shape='ovr', break_ties=False, random_state=None)

model.fit(X,y)


SVC(class_weight='balanced')

In [45]:
y_pred = model.predict(X)
conf_matrix = confusion_matrix(y, y_pred)
#accuracy_score = accuracy_score(y, y_pred)
#print(accuracy_score)
conf_matrix


array([[25,  3],
       [ 1,  7]])